In [17]:
# Process it all into a single file
import glob
with open("./dataset.tsv", "w") as out:
    out.write("Temperature(C)\tDoping(%)\tTime\tWavelength\tIntensity\n")
    for data in glob.glob('fluorescence_data/*.txt'):
        with open(data, 'r') as f:
            filename = data.split('/')[-1].split('.')[0]
            T = filename.split('-')[0]
            dop = filename.split('-')[1]
            t = filename.split('-')[2]
            if dop != "NPL":
                setup = [T, dop, t]
                for line in f:
                    new_line = setup + line.split()
                    out.write("\t".join(new_line)+"\n")
        

In [18]:
# Process it all into a single file filtering just the peak values
import glob
with open("./peak_dataset.tsv", "w") as out:
    out.write("Temperature(C)\tDoping(%)\tTime\tWavelength\n")
    for data in glob.glob('fluorescence_data/*.txt'):
        with open(data, 'r') as f:
            filename = data.split('/')[-1].split('.')[0]
            T = filename.split('-')[0]
            dop = filename.split('-')[1]
            t = filename.split('-')[2]
            if dop != "NPL":
                setup = [T, dop, t]
                best_intensity = -1
                out_line = ""
                for line in f:
                    new_line = setup + line.split()
                    if float(new_line[4]) > best_intensity:
                        best_intensity = float(new_line[4])
                        out_line = new_line
                out.write("\t".join(out_line[:-1])+"\n")
                
        